In [1]:

# coding: utf-8

# In[ ]:

import falcon
import sys, os

from fft import fft, ifft
from ip import ipFFT, inBufferFFT, ipiFFT, inBufferiFFT
from get_vector import get_fft1024, get_ifft1024, get_fft512, get_ifft512
from time import time
from math import log2

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'

def test_fft(n):
    print("=================================================")
    print("           *-------------------*")
    print(f"           |   For n = {n:>4}    |")    
    print("           *-------------------*")
    print("Start comparing the [FFT] IP and function...")

    f_in = [0 for i in range(n)]
    # Read the Input
    with open("FFT_in.txt", "r") as file:
        for i in range(n):
            f_in[i] = float(file.readline())
    # Start the Python Function
    start_function = time()
    ans = fft(f_in)
    end_function = time()

    for i in range(n):
        inBufferFFT[i] = f_in[i]
    
    logn = int(log2(n))
    print(f"logn = {logn}")
    ipFFT.write(0x1C, logn) # Coefficient
    
    # Start the FFT IP
    start_ip = time()
    ipFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipFFT.read(0x00) & 0x4) == 0x0:
        continue
    end_ip = time()
    
    if (n == 1024):
        ip_out = get_fft1024(inBufferFFT)
    elif (n == 512):
        ip_out = get_fft512(inBufferFFT)


    # Compare
    error_cnt = 0
    for i in range(len(ans)):
        r = 7
        # Round to the 7th digit after the decimal point
        ip_real = round(ip_out[i].real, r)
        ans_real = round(ans[i].real, r)
        ip_imag = round(ip_out[i].imag, r)
        ans_imag = round(ans[i].imag, r)
        f = complex(ip_real, ip_imag)
        a = complex(ans_real, ans_imag)
        if (f != a):
            print(f"Expect: {ans[i]}, actual: {ip_out[i]}")
            error_cnt += 1
    if (error_cnt != 0):
        print(f"Failed!!, there are {error_cnt+1} errors")
    else:
        print("Passed!!")

    print(f"{'FFT python execution time:':20} {(end_function - start_function):>.10f} sec")
    print(f"{'FFT IP execution time:':20}     {(end_ip - start_ip):>.10f} sec")
    print(f"Acceleration rate: {round((end_function - start_function) / (end_ip - start_ip), 2)}")
    print("-------------------------------------------------")


    print("Start comparing the [iFFT] IP and function")

    ipiFFT.write(0x1C, logn) # Coefficient

    if (n == 1024):
        f_in = get_ifft1024(ans) # Re-construct the order of the FFT output
    elif (n == 512):
        f_in = get_ifft512(ans)

    # Start the Python Function
    start_function = time()
    ans = ifft(ans) # upload the golden to the output of iFFT function
    end_function = time()

    for i in range(n):
        inBufferiFFT[i] = f_in[i]

    # Start the iFFT IP
    start_ip = time()
    ipiFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipiFFT.read(0x00) & 0x4) == 0x0:
        continue
    end_ip = time()

    # Compare
    error_cnt = 0
    for i in range(len(ans)):
        r = 10
        if (round(inBufferiFFT[i], r) != round(ans[i], r)):
            print(f"Expect: {ans[i]}, actual: {inBufferiFFT[i]}")
            error_cnt += 1
    if (error_cnt != 0):
        print(f"Failed!!, there are {error_cnt+1} errors")
    else:
        print("Passed!!")

    print(f"{'iFFT python execution time:':20} {(end_function - start_function):>.10f} sec")
    print(f"{'iFFT IP execution time:':20}     {(end_ip - start_ip):>.10f} sec")
    print(f"Acceleration rate: {round((end_function - start_function) / (end_ip - start_ip), 2)}")

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")

    test_fft(1024)
    test_fft(512)

    print("=================================================")
    print("Exit process")
    


Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
           *-------------------*
           |   For n = 1024    |
           *-------------------*
Start comparing the [FFT] IP and function...
logn = 10
Passed!!
FFT python execution time: 0.0279724598 sec
FFT IP execution time:     0.0020098686 sec
Acceleration rate: 13.92
-------------------------------------------------
Start comparing the [iFFT] IP and function
Passed!!
iFFT python execution time: 0.0315771103 sec
iFFT IP execution time:     0.0035049915 sec
Acceleration rate: 9.01
           *-------------------*
           |   For n =  512    |
           *-------------------*
Start comparing the [FFT] IP and function...
logn = 9
Passed!!
FFT python execution time: 0.0126500130 sec
FFT IP execution time:     0.0009019375 sec
Acceleration rate: 14.03
---------------------------------------